In [1]:
import pandas as pd
import numpy as np
import pickle

## Clean the web scraped data.

In [2]:
with open('./data/web_scraped_dataframe.pickle','rb') as read_file:
    df = pickle.load(read_file)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2294 entries, 0 to 2293
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   No.                  2294 non-null   object
 1   Player               2228 non-null   object
 2   Age                  2294 non-null   object
 3   Pos                  2294 non-null   object
 4   G                    2294 non-null   object
 5   GS                   2294 non-null   object
 6   Wt                   2294 non-null   object
 7   Ht                   2294 non-null   object
 8   College/Univ         2294 non-null   object
 9   BirthDate            2294 non-null   object
 10  Yrs                  2294 non-null   object
 11  AV                   2294 non-null   object
 12  Drafted (tm/rnd/yr)  2294 non-null   object
 13  Salary               2294 non-null   object
 14  Team                 2294 non-null   object
 15  Record               2294 non-null   object
 16  Player

In [4]:
df.head()

,No.,Player,Age,Pos,G,GS,Wt,Ht,College/Univ,BirthDate,Yrs,AV,Drafted (tm/rnd/yr),Salary,Team,Record,Player
0,97,Mario Addison,33,DE,15,7,260,6-3,Troy,9/6/1987,9,4,,"$5,250,000",buf,13-3-0,NaN
1,17,Josh Allen,24,QB,16,16,237,6-5,Wyoming,5/21/1996,2,18,Buffalo Bills / 1st / 7th pick / 2018,"$750,000",buf,13-3-0,NaN
2,5,Matt Barkley,30,QB,5,0,234,6-2,USC,9/8/1990,7,1,Philadelphia Eagles / 4th / 98th pick / 2013,"$1,500,000",buf,13-3-0,NaN
3,2,Tyler Bass,23,K,16,0,183,5-10,Georgia Southern,2/14/1997,Rook,3,Buffalo Bills / 6th / 188th pick / 2020,"$610,000",buf,13-3-0,NaN
4,71,Ryan Bates,23,OL,16,0,302,6-4,Penn St.,2/14/1997,1,1,,"$675,000",buf,13-3-0,NaN


In [5]:
# Make a function to convert elements to numeric.

def make_num_exception(val):
    """
    Convert elements to numeric.
    If NOT able to convert, return None but print the element value.
    Can then go back and change as needed.
    """
    try:
        return float(val)
    except ValueError:
        print("Error found with entry: " + val)
        return None
    
def series_to_num(series):
    """
    Convert pd series to numeric from string.
    """
    series = series.replace(r'^\s*$', np.nan, regex=True)
    series = series.apply(make_num_exception)
    return series

In [6]:
# Columns to numeric.
df["Age"] = series_to_num(df["Age"])

df["G"] = series_to_num(df["G"])

df["GS"] = series_to_num(df["GS"])

df["Wt"] = series_to_num(df["Wt"])

df["AV"] = series_to_num(df["AV"])

df["Salary"] = df["Salary"].map(lambda x: x.replace("$","").replace(",", ""))
df["Salary"] = series_to_num(df["Salary"])

In [7]:
def ht_conversion(string):
    """
    Convert heights given as "ft-in" to numeric inches.
    """
    if "-" not in string:
        return None
    ht_lst = string.split("-")
    return (float(ht_lst[0]) * 12) + float(ht_lst[1])


In [8]:
# Height to inches (numeric).
df["Ht"] = df["Ht"].apply(lambda x: ht_conversion(x))

In [9]:
# Record to win percentage.
df["Record"] = df["Record"].apply(lambda x: float(x.split("-")[0])/16)

In [10]:
# Convert years in the NFL to numeric (Rookies = 0).

df["Yrs"] = df["Yrs"].apply(lambda x: 0 if x == "Rook" else x)
df["Yrs"] = series_to_num(df["Yrs"])

In [11]:
# Combine some position descriptions.

df.replace({"Pos": {"LS,TE":"TE", "QB/TE":"TE", "WR/CB":"WR", "WR/RB":"WR"}}, inplace=True)

In [12]:
# Create general positions (offense, defense, special teams).
dct_gen_pos = {
    "offense": ["C", "RB", "FB", "G", "LG", "OT", "LT", "TE", "QB", "WR", "HB", "OL", 
                "T", "OG"],
    "defense": ["CB", "DE", "DT", "LB", "ILB", "MLB", "NT", "OLB", "S", "FS", "SS", "DL", 
                "DB", "RILB", "LILB", "LOLB", "EDGE"],
    "spec_t": ["K", "KR", "LS", "P"]
}


df["general_pos"] = np.nan

for k,v in dct_gen_pos.items():
    df.loc[df["Pos"].isin(v), "general_pos"] = k

In [13]:
# Create positional groups.

dct_pos_group = {
    # Defense
    "def_line": ["NT", "DT", "DE", "DL"],
    "linebackers": ["ILB", "MLB", "RILB", "LILB", "LOLB", "LB", "EDGE", "OLB"],
    "secondary": ["CB", "S", "FS", "SS", "DB"],
    # Offense
    "off_line": ["LG", "LT", "C", "T", "G", "OT", "OL", "OG"],
    "wr_te": ["WR", "TE"],
    "QB": ["QB"],
    "rb_fb": ["RB", "FB", "HB"],
    # Special Teams
    "spec_t": ["K", "KR", "LS", "P"]
}

df["pos_group"] = np.nan

for k,v in dct_pos_group.items():
    df.loc[df["Pos"].isin(v), "pos_group"] = k

In [14]:
# Create categorical data for starter, irregular-starter, non-starter

conditions = [
    (df["GS"] == 0), 
    (df["GS"] > 0) & (df["GS"] < 10),
    (df["GS"] >= 10) 
]

values = ["non-starter", "irreg_starter", "starter"]

df["starter"] = np.select(conditions, values)

In [15]:
# Calculate BMI.

df["BMI"] = round((df["Wt"] / (df["Ht"] **2)) *703, 1)

In [16]:
# Calculate where players stand in terms of height/weight compared to those in the same position.

df["ht_pos_mean"] = df.groupby(["Pos"])["Ht"].transform('mean')
df["ht_pos"] = np.where(df["ht_pos_mean"] < df["Ht"], "Under", "Equal_Above")

df["wt_pos_mean"] = df.groupby(["Pos"])["Wt"].transform('mean')
df["wt_pos"] = np.where(df["wt_pos_mean"] < df["Wt"], "Under", "Equal_Above")

In [17]:
# Drop unwanted columns and rename as needed.
df.columns = [column.strip() for column in df.columns]

df = df.drop(columns=["ht_pos_mean", "wt_pos_mean", "No.", "BirthDate", "Drafted (tm/rnd/yr)", "Player"])

df = df.rename(columns={"College/Univ":"College"})

In [18]:
# Have enough folks that I'm willing to drop those missing some variables for this analysis.

df.dropna(inplace=True)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1935 entries, 0 to 2292
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          1935 non-null   float64
 1   Pos          1935 non-null   object 
 2   G            1935 non-null   float64
 3   GS           1935 non-null   float64
 4   Wt           1935 non-null   float64
 5   Ht           1935 non-null   float64
 6   College      1935 non-null   object 
 7   Yrs          1935 non-null   float64
 8   AV           1935 non-null   float64
 9   Salary       1935 non-null   float64
 10  Team         1935 non-null   object 
 11  Record       1935 non-null   float64
 12  general_pos  1935 non-null   object 
 13  pos_group    1935 non-null   object 
 14  starter      1935 non-null   object 
 15  BMI          1935 non-null   float64
 16  ht_pos       1935 non-null   object 
 17  wt_pos       1935 non-null   object 
dtypes: float64(10), object(8)
memory usage: 287.2+ K

In [20]:
with open('./data/cleaned_dataframe.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)